第一题：在服务器上使⽤Docker 部署Open WebUI ，并下载DeepSeek-R1:8B 推理模型（若只有CPU，使⽤DeepSeek-R1:1.5B 模型），⽣成蓝⽛降噪⽿机的⼩红书⽂案。

![rednote](https://raw.githubusercontent.com/Switch-vov/deepseek-quickstart/refs/heads/main/deepseek/ollama/ollama_webui_generate_rednote.png)

第二题：将上⼀节课的⼩红书爆款⽂案助⼿项⽬中的DeepSeek API 调⽤，改为私有化部署DeepSeek-R1 模型，实现数据隐私保护。

In [9]:
# 在宿主机直接通过openai的形式访问
# 参考api文档：https://github.com/ollama/ollama/blob/main/docs/openai.md
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    # required but ignored
    api_key='ollama',
)

In [10]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

In [15]:
import json
import re


def generate_rednote(product_name: str, tone_style: str = "活泼甜美") -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。

    Args:
        product_name (str): 要生成文案的产品名称。
        tone_style (str): 文案的语气和风格，如"活泼甜美"、"知性"、"搞怪"等。
    Returns:
        str: 生成的爆款文案（JSON 格式字符串）。
    """

    print(f"\n🚀 启动小红书文案生成助手，产品：{product_name}，风格：{tone_style}\n")

    # 存储对话历史，包括系统提示词和用户请求
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",
         "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。"}
    ]

    response = client.chat.completions.create(
        model="deepseek-r1:8b",
        messages=messages,
    )
    response_message = response.choices[0].message
    # **ReAct模式：处理工具调用**
    if response_message.content:  # 如果模型直接返回内容（通常是最终答案）
        print(f"[模型生成结果] {response_message.content}")

        # --- START: 添加 JSON 提取和解析逻辑 ---
        content = response_message.content.split('</think>')[1]
        json_string_match = re.search(r"```json\s*(\{.*\})\s*```", content, re.DOTALL)
        if json_string_match:
            extracted_json_content = json_string_match.group(1)
            try:
                final_response = json.loads(extracted_json_content)
                print("Agent: 任务完成，成功解析最终JSON文案。")
                return json.dumps(final_response, ensure_ascii=False, indent=2)
            except json.JSONDecodeError as e:
                print(f"Agent: 提取到JSON块但解析失败: {e}")
                print(f"尝试解析的字符串:\n{extracted_json_content}")
                messages.append(response_message)  # 解析失败，继续对话
        else:
            # 如果没有匹配到 ```json 块，尝试直接解析整个 content
            try:
                final_response = json.loads(response_message.content)
                print("Agent: 任务完成，直接解析最终JSON文案。")
                return json.dumps(final_response, ensure_ascii=False, indent=2)
            except json.JSONDecodeError:
                print("Agent: 生成了非JSON格式内容或非Markdown JSON块，可能还在思考或出错。")
                messages.append(response_message)  # 非JSON格式，继续对话
        # --- END: 添加 JSON 提取和解析逻辑 ---
    else:
        print("Agent: 未知响应，可能需要更多交互。")
    return "未能成功生成文案。"

In [16]:
# 测试案例 1: 深海蓝藻保湿面膜
product_name_1 = "深海蓝藻保湿面膜"
tone_style_1 = "活泼甜美"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：深海蓝藻保湿面膜，风格：活泼甜美

[模型生成结果] <think>
首先，任务是为产品「深海蓝藻保湿面膜」生成小红书爆款文案。使用 Thought-Action-Observation 模式：

- **Thought**: 思考并收集足够的信息。

  - 我需要理解产品和需求。
    - 产品：深海蓝藻保湿面膜。这是一个护肤产品，主打保湿、深层清洁或提供海洋成分的益处（如蓝藻）。
    - 需求：语气活泼甜美，包含标题、正文、至少5个相关标签和5个表情符号。

- **Action**: 基于思考创建文案。

  - 文案风格要活泼、真诚、富有感染力。这意味着要用亲切的语言、感叹词、故事性元素来吸引读者。
    - 标题：应 catchy，使用 emojis 来增加吸引力。
    - 正文：分隔成自然段落，强调产品卖点，如保湿效果、成分（深海蓝藻）、使用感受，并加入互动元素（比如让读者分享自己的经验）。
    - 5个相关标签：基于产品关键词，选择流行的小红书标签。示例可能包括 #护肤 #面膜推荐 等。
    - 表情符号：至少5个，放在正文合适的位置来增强情感。

- **Observation**: 输出时要高互动、高转化的笔记文案，并以 JSON 格式输出。

现在，收集信息：

1. **产品卖点**：
   - "深海蓝藻"：一种富含营养和保湿成分的海洋生物。
   - 保湿面膜：可能强调补水、锁水、提亮肤色等功效。
   - 假设用户需求未提供更多细节，所以我需基于一般知识。

2. **小红书风格**：
   - 活泼甜美：使用可爱的语气词如“宝贝”、“哇塞”，感叹号。避免正式语言，加入个人故事和感受。
   - 高互动：鼓励评论、分享，比如问读者是否用过类似产品。
   - 高转化：突出优惠、独特卖点（USP），或强烈建议尝试。

3. **关键词 brainstorming**：
   - 标题选项："深海蓝藻保湿面膜试用报告！宝贝们冲鸭~" 或 "震惊！这个深海蓝藻面膜让我皮肤水当真✨"
   - 正文：介绍自己使用经历，成分益处（如抗衰老、净化肌肤），保湿效果的前后对比。
     - 段落1：引起注意 – 描述打开包装或初次使用的兴奋感。
     - 段落2：卖点 – 解释深海蓝藻的好处

In [18]:
# 测试案例 2: 美白精华
product_name_2 = "美白精华"
tone_style_2 = "知性温柔"
result_2 = generate_rednote(product_name_2, tone_style_2)

print("\n--- 生成的文案 2 ---")
print(result_2)


🚀 启动小红书文案生成助手，产品：美白精华，风格：知性温柔

[模型生成结果] <think>
首先，任务是根据产品「美白精华」生成一篇小红书爆款文案。用户指定语气要知性温柔。

关键元素：
- 标题：需要吸引人、引人入胜。
- 正文：高互动、高转化的笔记文案，风格活泼、真诚、富有感染力。
- 相关标签：至少5个，并用#开头。
- 表情符号：至少5个。
- 输出格式：JSON格式，包含title, body, hashtags, emojis。hashtags应该是数组如 ["#标签1", "#标签2"]，emojis是数组如 ["✨", "🔥"]。

模式'Thought-Action-Observation'：
1. **Thought**: 先思考并收集足够的信息。
   - 作为专家，我需要考虑美白精华的卖点：美白、提亮肤色、淡化痘印等。结合最新潮流，比如强调自然效果、科技成分或用户评价。
   - 文案风格：知性温柔，活泼真诚富有感染力。这意味着语言要亲切、软语调，避免过于强硬推销。
2. **Action**: 基于思考创作文案。

内容构思：
- 小红书平台的特点：偏向美妆、生活方式的分享，语气女性化、日常化。
- 标题想法：应该温柔吸引人，比如"我的美白秘诀：这支精华让我重获新生"或"知性优雅的选择 - 精华美白之旅"。
- 正文：结构可以是个人故事 + 产品体验。开头引入问题（如肤色不均），分享使用经历、效果、感受。加入互动元素（如提问读者）。语气要知性和温柔，所以用词柔和，焦点在自然美和内在自信上。
  - 示例正文：
    "亲爱的们，你们有没有发现自己的皮肤最近总是暗沉无光？😅 我也曾经是一个为肤色烦恼的人呢... 后来我尝试了这款美白精华，它真的改变了我的人生！✨"

- 标签：至少5个相关标签。主题是美白精华，所以标签如：
  - "#美白精华"
  - "#护肤心得"
  - "#肌肤护理"
  - "#淡斑美白"
  - "#美丽分享"（或其他流行标签）

- 表情符号：至少5个。需要生动、适合文案语气的emoji。建议与美容相关或情感相关，如💫, 😊, 💄等。

确保高互动和转化：
- 高互动：鼓励评论，提问读者。
- 高转化：自然过渡到购买或关注，但要真诚不含糊。例如："这款精华真的太好用了！快来试试吧~" 或分享成分和效果来

In [19]:
import json


def format_rednote_for_markdown(json_string: str) -> str:
    """
    将 JSON 格式的小红书文案转换为 Markdown 格式，以便于阅读和发布。

    Args:
        json_string (str): 包含小红书文案的 JSON 字符串。
                           预计格式为 {"title": "...", "body": "...", "hashtags": [...], "emojis": [...]}

    Returns:
        str: 格式化后的 Markdown 文本。
    """
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"错误：无法解析 JSON 字符串 - {e}\n原始字符串：\n{json_string}"

    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    # 表情符号通常已经融入标题和正文中，这里可以选择是否单独列出
    # emojis = data.get("emojis", [])

    # 构建 Markdown 文本
    markdown_output = f"## {title}\n\n"  # 标题使用二级标题

    # 正文，保留换行符
    markdown_output += f"{body}\n\n"

    # Hashtags
    if hashtags:
        hashtag_string = " ".join(hashtags)  # 小红书标签通常是空格分隔
        markdown_output += f"{hashtag_string}\n"

    # 如果需要，可以单独列出表情符号，但通常它们已经包含在标题和正文中
    # if emojis:
    #     emoji_string = " ".join(emojis)
    #     markdown_output += f"\n使用的表情：{emoji_string}\n"

    return markdown_output.strip()  # 去除末尾多余的空白

In [20]:
# 调用格式化函数
markdown_note = format_rednote_for_markdown(result_1)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

--- 格式化后的小红书文案 (Markdown) ---
## 海底捞到陆地？深海蓝藻保湿面膜让我皮肤水嫩到发光✨

Hi宝贝们～今天来安利一款超级清爽的深海蓝藻保湿面膜！🌊 先给大家科普一下，这款面膜是用纯天然深海蓝藻提取物制造的，富含藻类精华和超强抗氧化力，简直是海洋护肤界的宝藏！😍 我最近天气干燥皮肤总是紧绷绷的，一到晚上就感觉暗沉没精气。某天我rua了一盒这个面膜（原谅我的直男口癖），敷上去才15分钟感觉整个脸都被净化了～💦 质地轻薄不黏腻，洗完脸后摸起来滑得像剥了壳的鸡蛋，超级享受！用了一个星期，最明显的变化是肤色均匀了好多，暗沉悄悄不见啦。细节观察：敷前面膜有点凉意镇静肌肤，敷后面皮肤吸收超快～而且它还是温和型的，干皮敏皮亲妈推荐！水嫩心得分享时间：我每周用两次，现在毛孔超级细腻，弹润度拉满。如果你们之前试过类似产品欢迎在评论区交♂️流啊～有没有更好的保湿秘籍？来喽姐妹们一起进步吧！💕

#保湿面膜 #深海蓝藻护肤 #美白补水 #美容心得 #晚间护理


In [22]:
# 调用格式化函数
markdown_note = format_rednote_for_markdown(result_2)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

--- 格式化后的小红书文案 (Markdown) ---
## ✨我的肌肤蜕变journey：温和美白精华让我绽放知性美

亲爱的们～💖 种草分享一款我最近用到的'圣品'，它是某品牌的明星美白精华！🌙 前几天镜子里看到自己的肤色总是暗沉不均，忍不住开始思考护肤的秘密啊...经过这段时间的尝试，这款产品真的让我惊喜连连~ 它富含维生素C导体和果酸成分，温和不刺激，帮助均匀肤色、淡化痘印。刚开始使用时感觉肌肤轻盈水润，渐渐地，整体提亮了！✨ 肤质也越来越透亮光滑，像从内而外地明亮起来。你们觉得这种天然的护理方式靠谱吗？😉 其实每个人的皮肤都是独一无二的，找到适合自己的才是王道。如果你也还在为肤色烦恼，别急着焦虑哟~ 轻脆入手试试吧！🌱 希望我的分享能帮到你们～一起变美不是梦！💖

#美白精华 #护肤心得 #淡斑护理 #肌肤焕白 #美丽日常
